system requirements
1. activate gedi env, cuda 11.7
2. install tinycudann

In [1]:
import torch
import numpy as np
import open3d as o3d
import torch.nn.functional as F
from gedi import GeDi
import torch.nn as nn
import math
import os

from models.fields import FeatureField
from pyhocon import ConfigFactory

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

"""script for extract DINO feature from rendered image and geometric features from pointcloud, visualizing feature matching results
"""
def sample_point_cloud_from_mesh(mesh_path, number_of_points=100000,save_to_file=True):
    mesh = o3d.io.read_triangle_mesh(mesh_path)
    
    # Check if the mesh has vertex normals, if not, compute them
    if not mesh.has_vertex_normals():
        mesh.compute_vertex_normals()
    
    # Sample points uniformly from the mesh
    pcd = mesh.sample_points_uniformly(number_of_points=number_of_points)

    if save_to_file:
        pcd_fname = mesh_path.replace("glb","pcd")
        if not os.path.exists(pcd_fname):
            print(f'cache extracted pcd file to {pcd_fname}')
            o3d.io.write_point_cloud(pcd_fname, pcd)
    
    return pcd

def load_pointcloud(pcd_fname):
    if os.path.exists(pcd_fname):
        print(f"read cached pcd file from {pcd_fname}")
        pcd = o3d.io.read_point_cloud(pcd_fname)
    else:
        print("can't find cached pcd file, read from mesh file")
        mesh_path = pcd_fname.replace("pcd","glb")
        pcd = sample_point_cloud_from_mesh(mesh_path)
    return pcd

def visualize_point_cloud(pcd):
    # Visualize the point cloud
    o3d.visualization.draw_geometries([pcd])


def load_checkpoint(checkpoint_fname):
    # read network config
    conf_path = './confs/wmask.conf'
    f = open(conf_path)
    conf_text = f.read()
    conf_text = conf_text.replace('CASE_NAME', 'owl') #TODO case name as input
    f.close()
    conf = ConfigFactory.parse_string(conf_text)

    # load feature field
    checkpoint = torch.load(os.path.join(checkpoint_fname), map_location='cuda')
    feature_network = FeatureField(**conf['model.feature_field']).to('cuda')
    # self.nerf_outside.load_state_dict(checkpoint['nerf'])
    # self.sdf_network.load_state_dict(checkpoint['sdf_network_fine'])
    # self.deviation_network.load_state_dict(checkpoint['variance_network_fine'])
    # self.color_network.load_state_dict(checkpoint['color_network_fine'])
    feature_network.load_state_dict(checkpoint['feature_network'])
    # self.optimizer_geometry.load_state_dict(checkpoint['optimizer-geometry'])
    # self.optimizer_feature.load_state_dict(checkpoint['optimizer-feature'])

    # iter_step = checkpoint['iter_step']
    print(f"loaded checkpoint from {checkpoint_fname}")
    return feature_network


def visualize_point_cloud(pcd):
    # Visualize the point cloud
    o3d.visualization.draw_geometries([pcd])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def extract_geo_feature(pcd=None, config=None):
    if config == None:
        config = {
          'dim': 32,                                            # descriptor output dimension
          'samples_per_batch': 500,                             # batches to process the data on GPU
          'samples_per_patch_lrf': 4000,                        # num. of point to process with LRF
          'samples_per_patch_out': 512,                         # num. of points to sample for pointnet++
          'r_lrf': .5,                                          # LRF radius
          'fchkpt_gedi_net': 'data/chkpts/3dmatch/chkpt.tar',   # path to checkpoint
          'device':'cpu',
        }  
    voxel_size = .01
    patches_per_pair = 5000

    # initialising class
    gedi = GeDi(config=config)

    # getting a pair of point clouds
    if pcd==None: # if not passing pcd directly, read from local storage
        pcd0 = o3d.io.read_point_cloud('data/assets/threed_match_7-scenes-redkitchen_cloud_bin_0.ply')
    else:
        pcd0 = pcd

    pcd0.paint_uniform_color([1, 0.706, 0])

    # estimating normals (only for visualisation)
    pcd0.estimate_normals()

    # randomly sampling some points from the point cloud
    inds0 = np.random.choice(np.asarray(pcd0.points).shape[0], patches_per_pair, replace=False) # 5000

    pts0 = torch.tensor(np.asarray(pcd0.points)[inds0]).float() # 5000 x 3 

    # applying voxelisation to the point cloud
    pcd0 = pcd0.voxel_down_sample(voxel_size)

    # pcd in tensor
    _pcd0 = torch.tensor(np.asarray(pcd0.points)).float()

    # computing descriptors
    pcd0_desc = gedi.compute(pts=pts0, pcd=_pcd0) # 5000x32

    return pcd0_desc, pts0

# is scale kept consist from MESH to pcd then goes back? are geometry feature point and dino feature point the same point?
def extract_DINO_feature_from_model(model_ckpt, pts):
    model = load_checkpoint(model_ckpt)
    return model(pts)



### Here setup the case name

In [3]:
casename1 = 'cup1'
casename2 = 'cup2'
iteration = '3000'

In [4]:

# pcd_path_source = "./exp/neus/mixer/meshes/mixer_5000.pcd" 
pcd_path_source1 = f"./exp/neus/{casename1}/meshes/{casename1}_{iteration}.pcd"
pcd_source1= load_pointcloud(pcd_path_source1)
geo_features1, sampled_pts1 = extract_geo_feature(pcd_source1) # tensor on cpu, bc some operation dosen't support cuda

# move tensor to GPU 
geo_features1 = geo_features1.to(device)
sampled_pts1 = sampled_pts1.to(device)

# extract DINO feature
ckpt = f'./exp/neus/{casename1}/checkpoints/ckpt_00{iteration}.pth'
DINO_features1 = extract_DINO_feature_from_model(ckpt,sampled_pts1)

features1 = torch.cat((geo_features1,DINO_features1),1) # combined feature
# features = DINO_features # DINO only


read cached pcd file from ./exp/neus/cup1/meshes/cup1_3000.pcd
loaded checkpoint from ./exp/neus/cup1/checkpoints/ckpt_003000.pth


In [5]:
# extract features from second pointcloud
# pcd_path_source2 = "./exp/neus/owl/meshes/owl.pcd" 
pcd_path_source2 = f"./exp/neus/{casename2}/meshes/{casename2}_{iteration}.pcd"

pcd_source2= load_pointcloud(pcd_path_source2)
geo_features2, sampled_pts2 = extract_geo_feature(pcd_source2) # tensor on cpu, bc some operation dosen't support cuda

# move tensor to GPU 
geo_features2 = geo_features2.to(device)
sampled_pts2 = sampled_pts2.to(device)

# extract DINO feature
ckpt = f'./exp/neus/{casename2}/checkpoints/ckpt_005000.pth'
DINO_features2 = extract_DINO_feature_from_model(ckpt,sampled_pts2)

features2 = torch.cat((geo_features2,DINO_features2),1) # combine features
# features2 = DINO_features2 # DINO only

read cached pcd file from ./exp/neus/cup2/meshes/cup2_3000.pcd
loaded checkpoint from ./exp/neus/cup2/checkpoints/ckpt_005000.pth


### feature matching


In [15]:
# target_indxs = [np.random.randint(0,len(sampled_pts1))  for i in range(10)] # randomly choose some pts for finding corresponding

# indx_1 = []
# indx_2 = []
# for ind in target_indxs:
#     target_pt1 = sampled_pts1[ind]
#     target_feature1 = features1[ind].reshape(1,-1)

#     features1_tmp = target_feature1.repeat(features2.shape[0],1) # generate bunch 

#     # Normalize the features to get cosine similarity
#     features1_norm = F.normalize(features1_tmp, p=2, dim=1)
#     features2_norm = F.normalize(features2, p=2, dim=1)

#     # Compute cosine similarity between all pairs
#     cosine_sim = torch.mm(features1_norm, features2_norm.t())

#     # Flatten the cosine similarity matrix and get the top similarity indices
#     num_rank = 10
#     cosine_sim_flat = cosine_sim.view(-1)
#     topk_values, topk_indices = torch.topk(cosine_sim_flat, num_rank, largest=True)

#     # Manually convert flat indices to 2D indices
#     n_rows = cosine_sim.size(0)
#     topk_indices_2d = [(idx // n_rows, idx % n_rows) for idx in topk_indices]

#     print("Top 10 closest pairs (indices and cosine similarities):")
#     for i in range(num_rank):
#         idx1, idx2 = topk_indices_2d[i]
#         print(f"Pair {i+1}: (features1[{idx1}], features2[{idx2}]), Cosine Similarity: {topk_values[i].item()}")
#         indx_1.append(ind)
#         indx_2.append(idx2)


Top 10 closest pairs (indices and cosine similarities):
Pair 1: (features1[7], features2[4674]), Cosine Similarity: 0.629497230052948
Pair 2: (features1[6], features2[4674]), Cosine Similarity: 0.629497230052948
Pair 3: (features1[4], features2[4674]), Cosine Similarity: 0.629497230052948
Pair 4: (features1[5], features2[4674]), Cosine Similarity: 0.629497230052948
Pair 5: (features1[1], features2[4674]), Cosine Similarity: 0.629497230052948
Pair 6: (features1[0], features2[4674]), Cosine Similarity: 0.629497230052948
Pair 7: (features1[2], features2[4674]), Cosine Similarity: 0.629497230052948
Pair 8: (features1[3], features2[4674]), Cosine Similarity: 0.629497230052948
Pair 9: (features1[8], features2[4674]), Cosine Similarity: 0.629497230052948
Pair 10: (features1[9], features2[4674]), Cosine Similarity: 0.629497230052948
Top 10 closest pairs (indices and cosine similarities):
Pair 1: (features1[7], features2[2529]), Cosine Similarity: 0.7816359996795654
Pair 2: (features1[6], featu

In [6]:
target_indxs = [np.random.randint(0,len(sampled_pts1))  for i in range(10)] # randomly choose some pts for finding corresponding

indx_1 = []
indx_2 = []
num_rank = 2
for ind in target_indxs:

    features1_tmp = features1[ind]
    # # Normalize the features to get cosine similarity
    features1_norm = F.normalize(features1_tmp, p=2, dim=0)
    features2_norm = F.normalize(features2, p=2, dim=1)

    # Compute cosine similarity between all pairs
    cosine_sim = torch.matmul(features2_norm, features1_norm.t())

    # # Flatten the cosine similarity matrix and get the top similarity indice
    topk_values, topk_indices = torch.topk(cosine_sim, num_rank, largest=True)

    # Get the rows with the maximum dot product values
    max_rows = features2_norm[topk_indices]

    # If you need the values as well
    max_values = cosine_sim[topk_indices]

    print("Indices of rows with maximum dot product:", topk_indices)
    # print("Rows with maximum dot product:", max_rows)
    print("Maximum dot product values:", max_values)
    indx_1.append(ind)
    indx_2.append(topk_indices[0])
    indx_1.append(ind)
    indx_2.append(topk_indices[1])

Indices of rows with maximum dot product: tensor([2580, 1126], device='cuda:0')
Maximum dot product values: tensor([0.7148, 0.7148], device='cuda:0', grad_fn=<IndexBackward>)
Indices of rows with maximum dot product: tensor([4659, 2276], device='cuda:0')
Maximum dot product values: tensor([0.7245, 0.7203], device='cuda:0', grad_fn=<IndexBackward>)
Indices of rows with maximum dot product: tensor([2790, 3438], device='cuda:0')
Maximum dot product values: tensor([0.7562, 0.7541], device='cuda:0', grad_fn=<IndexBackward>)
Indices of rows with maximum dot product: tensor([3489, 4938], device='cuda:0')
Maximum dot product values: tensor([0.8135, 0.8119], device='cuda:0', grad_fn=<IndexBackward>)
Indices of rows with maximum dot product: tensor([4242,  701], device='cuda:0')
Maximum dot product values: tensor([0.8018, 0.7987], device='cuda:0', grad_fn=<IndexBackward>)
Indices of rows with maximum dot product: tensor([ 585, 4371], device='cuda:0')
Maximum dot product values: tensor([0.5321, 0

### visualization

In [7]:
# Create a new point cloud to hold the combined points for visualization
combined_points = np.vstack((sampled_pts1.cpu(), sampled_pts2.cpu()+1.0)) # some offset
combined_pcd = o3d.geometry.PointCloud()
combined_pcd.points = o3d.utility.Vector3dVector(combined_points)

# Extract the corresponding points for the top pairs
top_pairs_pts1 = np.array([sampled_pts1[idx].cpu().numpy() for idx in indx_1])
top_pairs_pts2 = np.array([sampled_pts2[idx].cpu().numpy()+1.0 for idx in indx_2]) # compensate for the offset

# Create lines connecting the top pairs
# lines = [[idx1.cpu().numpy(), (idx2 + len(sampled_pts)).cpu().numpy()] for idx1, idx2 in topk_indices_2d]
lines = [[i, i + len(top_pairs_pts1)] for i in range(len(top_pairs_pts1))]

# Visualize point clouds with lines between top pairs
colors = [[0, 0, 1] for _ in range(len(lines))]  # Blue lines

line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(np.vstack((top_pairs_pts1, top_pairs_pts2)))
line_set.lines = o3d.utility.Vector2iVector(lines)
line_set.colors = o3d.utility.Vector3dVector(colors)

spheres = []
radius = 0.02  # Set the radius for the spheres
for pt in np.vstack((top_pairs_pts1, top_pairs_pts2)):
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=radius)
    sphere.translate(pt)
    sphere.paint_uniform_color([1, 0, 0])  # Color the spheres red
    spheres.append(sphere)

# Visualize the point clouds and the lines
o3d.visualization.draw_geometries([combined_pcd, line_set] + spheres)

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


: 

-----------------
### past results


In [ ]:
# 5000 to 5000 code
# Normalize the features to get cosine similarity
features1_norm = F.normalize(features1, p=2, dim=1)
features2_norm = F.normalize(features2, p=2, dim=1)

# Compute cosine similarity between all pairs
cosine_sim = torch.mm(features1_norm, features2_norm.t())

# Flatten the cosine similarity matrix and get the top 10 indices
num_rank = 3
cosine_sim_flat = cosine_sim.view(-1)
topk_values, topk_indices = torch.topk(cosine_sim_flat, num_rank, largest=True)

# Manually convert flat indices to 2D indices
n_rows = cosine_sim.size(0)
topk_indices_2d = [(idx // n_rows, idx % n_rows) for idx in topk_indices]

print("Top 10 closest pairs (indices and cosine similarities):")
for i in range(num_rank):
    idx1, idx2 = topk_indices_2d[i]
    print(f"Pair {i+1}: (features1[{idx1}], features2[{idx2}]), Cosine Similarity: {topk_values[i].item()}")